# Preprocessing data

In [1]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import KNNImputer, IterativeImputer 
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [2]:
SEED = 42
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
train_merged = pd.read_csv("merged_train.csv")
test_merged = pd.read_csv("merged_test.csv")

In [4]:
train_merged.head(3)

,calendar_dt,store_id,target,calendar_dt_facts,fact_staff_value_lag_1,fact_load_factor_lag_1,num_available_couriers_lag_1,fact_num_orders_lag_1,fact_percent_lateness_lag_1,city_nm,store_lifetime_in_days,fact_staff_churn,flag_high_load_lag_1,marketing_costs_lag_1,fact_couriers_with_shifts_lag_1,predicted_staff_value,predicted_num_orders,predicted_load_factor
0,2025-11-03,000fade4-e8dc-11ed-b10a-08c0eb31fffb,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.00000,260.00000,4.96000
1,2025-11-10,000fade4-e8dc-11ed-b10a-08c0eb31fffb,1.00000,2025-11-03,1.00000,0.52632,10.00000,10.00000,NaN,Ульяновск,888.00000,1.00000,1.00000,368357734.51660,19.00000,6.00000,290.00000,5.06000
2,2025-11-17,000fade4-e8dc-11ed-b10a-08c0eb31fffb,4.00000,2025-11-10,8.00000,1.94118,13.00000,33.00000,69.56522,Ульяновск,895.00000,1.00000,1.00000,NaN,17.00000,7.00000,280.00000,4.86000


In [5]:
test_merged.head(3)

,store_id,calendar_dt,calendar_dt_facts,fact_staff_value_lag_1,fact_load_factor_lag_1,num_available_couriers_lag_1,fact_num_orders_lag_1,fact_percent_lateness_lag_1,city_nm,store_lifetime_in_days,fact_staff_churn,flag_high_load_lag_1,marketing_costs_lag_1,fact_couriers_with_shifts_lag_1,predicted_staff_value,predicted_num_orders,predicted_load_factor
0,000fade4-e8dc-11ed-b10a-08c0eb31fffb,2025-11-24,2025-11-17,8,2.40000,12,36,66.66667,Ульяновск,902.00000,4.00000,1,42545024115.87992,15.00000,8,320,4.66000
1,0022f1b0-b8f8-11ee-b10b-08c0eb31fffb,2025-11-24,2025-11-17,6,3.20000,7,32,68.75000,Набережные Челны,631.00000,0.00000,0,NaN,10.00000,5,230,6.47000
2,00440ac1-6a1d-11eb-85a3-1c34dae33151,2025-11-24,2025-11-17,9,3.36364,12,37,100.00000,Новосибирск,1693.00000,1.00000,1,2187863.74863,11.00000,9,340,4.47000


## Заполнение пропусков

In [6]:
class NanFiller:
    def __init__(self, lifetime_col='store_lifetime_in_days'):
        self.medians = {} # Сохранение медиан для числовых колонок
        self.modes = {}  # для флагов
        self.cat_fill = {} # Для категориальных
        self.lifetime_col = lifetime_col
        
    def fit(self, df, nan_cols=None, flag_cols=None, cat_cols=None):
        if nan_cols is None: nan_cols = []
        if flag_cols is None: flag_cols = []
        if cat_cols is None: cat_cols = []
        
        # Числовые (не флаги)
        for col in nan_cols:
            if col in df.columns:
                self.medians[col] = df[col].median()
        
        # Флаги (бинарные)
        for col in flag_cols:
            if col in df.columns:
                # Мода для флагов (0 или 1)
                mode_val = df[col].mode()
                self.modes[col] = mode_val[0] if not mode_val.empty else 0
        
        # Категориальные
        for col in cat_cols:
            if col in df.columns:
                self.cat_fill[col] = 'пусто'
        
        return self
    
    def transform(self, df):
        df_transformed = df.copy()
        is_new = df_transformed[self.lifetime_col].fillna(0) < 7
        
        # 1. Числовые колонки
        for col, median in self.medians.items():
            if col in df_transformed.columns:
                df_transformed.loc[is_new & df_transformed[col].isna(), col] = 0
                df_transformed.loc[~is_new & df_transformed[col].isna(), col] = median
        
        # 2. Флаги
        for col, mode_val in self.modes.items():
            if col in df_transformed.columns:
                # Новые магазины: флаг = 0 (не было высокой нагрузки)
                df_transformed.loc[is_new & df_transformed[col].isna(), col] = 0
                # Старые магазины: флаг = мода (чаще всего было 0 или 1)
                df_transformed.loc[~is_new & df_transformed[col].isna(), col] = mode_val
        
        # 3. Категориальные
        for col, fill_val in self.cat_fill.items():
            if col in df_transformed.columns:
                df_transformed[col] = df_transformed[col].fillna(fill_val).astype('category')
        
        return df_transformed
    
    def fit_transform(self, df, nan_cols, flag_cols, cat_cols):
        """Объединенный метод"""
        self.fit(df, nan_cols, flag_cols, cat_cols)
        return self.transform(df)

In [7]:
filler = NanFiller()

In [8]:
num_cols = [
    "fact_staff_value_lag_1", "fact_load_factor_lag_1", "num_available_couriers_lag_1", "fact_num_orders_lag_1", "fact_percent_lateness_lag_1",
    "store_lifetime_in_days", "fact_staff_churn", "marketing_costs_lag_1", "fact_couriers_with_shifts_lag_1"
]
flag_cols = ["flag_high_load_lag_1"]
cat_cols = ["city_nm"]

In [9]:
train_filled = filler.fit_transform(train_merged, num_cols, flag_cols, cat_cols)

In [10]:
train_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8220 entries, 0 to 8219
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   calendar_dt                      8220 non-null   object  
 1   store_id                         8220 non-null   object  
 2   target                           8220 non-null   float64 
 3   calendar_dt_facts                5763 non-null   object  
 4   fact_staff_value_lag_1           8220 non-null   float64 
 5   fact_load_factor_lag_1           8220 non-null   float64 
 6   num_available_couriers_lag_1     8220 non-null   float64 
 7   fact_num_orders_lag_1            8220 non-null   float64 
 8   fact_percent_lateness_lag_1      8220 non-null   float64 
 9   city_nm                          8220 non-null   category
 10  store_lifetime_in_days           8220 non-null   float64 
 11  fact_staff_churn                 8220 non-null   float64 
 12  flag_h

In [11]:
train_filled = train_filled.drop_duplicates()

In [12]:
cols_to_check = ['predicted_staff_value', 'predicted_num_orders', 'predicted_load_factor']
train_filled = train_filled.dropna(subset=cols_to_check)

In [13]:
train_filled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8219 entries, 0 to 8219
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   calendar_dt                      8219 non-null   object  
 1   store_id                         8219 non-null   object  
 2   target                           8219 non-null   float64 
 3   calendar_dt_facts                5762 non-null   object  
 4   fact_staff_value_lag_1           8219 non-null   float64 
 5   fact_load_factor_lag_1           8219 non-null   float64 
 6   num_available_couriers_lag_1     8219 non-null   float64 
 7   fact_num_orders_lag_1            8219 non-null   float64 
 8   fact_percent_lateness_lag_1      8219 non-null   float64 
 9   city_nm                          8219 non-null   category
 10  store_lifetime_in_days           8219 non-null   float64 
 11  fact_staff_churn                 8219 non-null   float64 
 12  flag_high_l

In [14]:
train_filled.head(3)

,calendar_dt,store_id,target,calendar_dt_facts,fact_staff_value_lag_1,fact_load_factor_lag_1,num_available_couriers_lag_1,fact_num_orders_lag_1,fact_percent_lateness_lag_1,city_nm,store_lifetime_in_days,fact_staff_churn,flag_high_load_lag_1,marketing_costs_lag_1,fact_couriers_with_shifts_lag_1,predicted_staff_value,predicted_num_orders,predicted_load_factor
0,2025-11-03,000fade4-e8dc-11ed-b10a-08c0eb31fffb,1.00000,NaN,0.00000,0.00000,0.00000,0.00000,0.00000,пусто,0.00000,0.00000,0.00000,0.00000,0.00000,7.00000,260.00000,4.96000
1,2025-11-10,000fade4-e8dc-11ed-b10a-08c0eb31fffb,1.00000,2025-11-03,1.00000,0.52632,10.00000,10.00000,80.00000,Ульяновск,888.00000,1.00000,1.00000,368357734.51660,19.00000,6.00000,290.00000,5.06000
2,2025-11-17,000fade4-e8dc-11ed-b10a-08c0eb31fffb,4.00000,2025-11-10,8.00000,1.94118,13.00000,33.00000,69.56522,Ульяновск,895.00000,1.00000,1.00000,431786064.68392,17.00000,7.00000,280.00000,4.86000


In [15]:
test_filled = filler.transform(test_merged)

In [16]:
test_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2438 entries, 0 to 2437
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   store_id                         2438 non-null   object  
 1   calendar_dt                      2438 non-null   object  
 2   calendar_dt_facts                2438 non-null   object  
 3   fact_staff_value_lag_1           2438 non-null   int64   
 4   fact_load_factor_lag_1           2438 non-null   float64 
 5   num_available_couriers_lag_1     2438 non-null   int64   
 6   fact_num_orders_lag_1            2438 non-null   int64   
 7   fact_percent_lateness_lag_1      2438 non-null   float64 
 8   city_nm                          2438 non-null   category
 9   store_lifetime_in_days           2438 non-null   float64 
 10  fact_staff_churn                 2438 non-null   float64 
 11  flag_high_load_lag_1             2438 non-null   int64   
 12  market

In [17]:
train_filled.to_csv("train_filled.csv", index=False)

In [18]:
test_filled.to_csv("test_filled.csv", index=False)

## Создание новых фичей

In [148]:
def build_new_features(df):
    new_df = df.copy(deep=True)
    new_df['date'] = pd.to_datetime(new_df['calendar_dt'])
    # Разница между прогнозом и реальностью прошлой недели
    new_df['staff_prediction_gap'] = new_df['predicted_staff_value'] - new_df['fact_staff_value_lag_1']
    new_df['orders_prediction_gap'] = new_df['predicted_num_orders'] - new_df['fact_num_orders_lag_1']

    # Сколько заказов на одного курьера в прошлой неделе
    new_df['past_productivity'] = new_df['fact_num_orders_lag_1'] / new_df['fact_staff_value_lag_1'].replace(0, 1)
    # Прогнозная производительность на эту неделю
    new_df['predicted_productivity'] = new_df['predicted_num_orders'] / new_df['predicted_staff_value'].replace(0, 1)

    new_df['month'] = new_df['date'].dt.month
    new_df['quarter'] = new_df['date'].dt.quarter
    
    new_df['load_factor_gap'] = new_df['predicted_load_factor'] - new_df['fact_load_factor_lag_1']
    new_df['churn_ratio'] = new_df['fact_staff_churn'] / new_df['fact_staff_value_lag_1'].replace(0, 1)
    new_df['orders_per_courier_gap'] = new_df['predicted_productivity'] - new_df['past_productivity']
    new_df['load_x_churn'] = new_df['predicted_load_factor'] * new_df['fact_staff_churn']
    new_df['staff_change_pct'] = (new_df['predicted_staff_value'] - new_df['fact_staff_value_lag_1']) / new_df['fact_staff_value_lag_1'].replace(0, 1)
    new_df['orders_change_pct'] = (new_df['predicted_num_orders'] - new_df['fact_num_orders_lag_1']) / new_df['fact_num_orders_lag_1'].replace(0, 1)
    new_df['tension_index'] = new_df['predicted_num_orders'] / new_df['num_available_couriers_lag_1'].replace(0, 1)
    new_df['marketing_efficiency'] = new_df['predicted_num_orders'] / new_df['marketing_costs_lag_1'].replace(0, 1)
    new_df['is_critical_late'] = (new_df['fact_percent_lateness_lag_1'] > 0.3).astype(int)  # если >30% опозданий
    new_df['is_low_staff'] = (new_df['predicted_staff_value'] < new_df['fact_staff_value_lag_1'] * 0.9).astype(int)

    new_df['load_factor_log'] = np.log1p(new_df['predicted_load_factor'])
    new_df['load_factor_sqrt'] = np.sqrt(new_df['predicted_load_factor'])
    new_df['load_factor_squared'] = new_df['predicted_load_factor'] ** 2
    new_df['load_factor_inv'] = 1 / (new_df['predicted_load_factor'] + 0.001)

    return new_df

In [149]:
train_processed = build_new_features(train_filled)

In [150]:
test_processed = build_new_features(test_filled)

In [151]:
train_processed.to_csv("train_processed.csv", index=False)

In [152]:
test_processed.to_csv("test_processed.csv", index=False)

## Удаление ненужных признаков для финального датасета для модели

In [153]:
def drop_unused_features(df, list_of_cols):
    new_df = df.copy(deep=True)
    return new_df.drop(columns=list_of_cols)

In [154]:
train_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8219 entries, 0 to 8219
Data columns (total 39 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   calendar_dt                      8219 non-null   object        
 1   store_id                         8219 non-null   object        
 2   target                           8219 non-null   float64       
 3   calendar_dt_facts                5762 non-null   object        
 4   fact_staff_value_lag_1           8219 non-null   float64       
 5   fact_load_factor_lag_1           8219 non-null   float64       
 6   num_available_couriers_lag_1     8219 non-null   float64       
 7   fact_num_orders_lag_1            8219 non-null   float64       
 8   fact_percent_lateness_lag_1      8219 non-null   float64       
 9   city_nm                          8219 non-null   category      
 10  store_lifetime_in_days           8219 non-null   float64       
 

In [155]:
train_processed.head(3)

,calendar_dt,store_id,target,calendar_dt_facts,fact_staff_value_lag_1,fact_load_factor_lag_1,num_available_couriers_lag_1,fact_num_orders_lag_1,fact_percent_lateness_lag_1,city_nm,...,staff_change_pct,orders_change_pct,tension_index,marketing_efficiency,is_critical_late,is_low_staff,load_factor_log,load_factor_sqrt,load_factor_squared,load_factor_inv
0,2025-11-03,000fade4-e8dc-11ed-b10a-08c0eb31fffb,1.00000,NaN,0.00000,0.00000,0.00000,0.00000,0.00000,пусто,...,7.00000,260.00000,260.00000,260.00000,0,0,1.78507,2.22711,24.60160,0.20157
1,2025-11-10,000fade4-e8dc-11ed-b10a-08c0eb31fffb,1.00000,2025-11-03,1.00000,0.52632,10.00000,10.00000,80.00000,Ульяновск,...,5.00000,28.00000,29.00000,0.00000,1,0,1.80171,2.24944,25.60360,0.19759
2,2025-11-17,000fade4-e8dc-11ed-b10a-08c0eb31fffb,4.00000,2025-11-10,8.00000,1.94118,13.00000,33.00000,69.56522,Ульяновск,...,-0.12500,7.48485,21.53846,0.00000,1,1,1.76815,2.20454,23.61960,0.20572


In [156]:
test_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2438 entries, 0 to 2437
Data columns (total 38 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   store_id                         2438 non-null   object        
 1   calendar_dt                      2438 non-null   object        
 2   calendar_dt_facts                2438 non-null   object        
 3   fact_staff_value_lag_1           2438 non-null   int64         
 4   fact_load_factor_lag_1           2438 non-null   float64       
 5   num_available_couriers_lag_1     2438 non-null   int64         
 6   fact_num_orders_lag_1            2438 non-null   int64         
 7   fact_percent_lateness_lag_1      2438 non-null   float64       
 8   city_nm                          2438 non-null   category      
 9   store_lifetime_in_days           2438 non-null   float64       
 10  fact_staff_churn                 2438 non-null   float64    

In [157]:
drop_cols = ["calendar_dt", "store_id", "calendar_dt_facts", "date"]

In [158]:
train_final = drop_unused_features(train_processed, drop_cols)

In [159]:
test_final = drop_unused_features(test_processed, drop_cols)

In [160]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8219 entries, 0 to 8219
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   target                           8219 non-null   float64 
 1   fact_staff_value_lag_1           8219 non-null   float64 
 2   fact_load_factor_lag_1           8219 non-null   float64 
 3   num_available_couriers_lag_1     8219 non-null   float64 
 4   fact_num_orders_lag_1            8219 non-null   float64 
 5   fact_percent_lateness_lag_1      8219 non-null   float64 
 6   city_nm                          8219 non-null   category
 7   store_lifetime_in_days           8219 non-null   float64 
 8   fact_staff_churn                 8219 non-null   float64 
 9   flag_high_load_lag_1             8219 non-null   float64 
 10  marketing_costs_lag_1            8219 non-null   float64 
 11  fact_couriers_with_shifts_lag_1  8219 non-null   float64 
 12  predicted_s

In [161]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2438 entries, 0 to 2437
Data columns (total 34 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   fact_staff_value_lag_1           2438 non-null   int64   
 1   fact_load_factor_lag_1           2438 non-null   float64 
 2   num_available_couriers_lag_1     2438 non-null   int64   
 3   fact_num_orders_lag_1            2438 non-null   int64   
 4   fact_percent_lateness_lag_1      2438 non-null   float64 
 5   city_nm                          2438 non-null   category
 6   store_lifetime_in_days           2438 non-null   float64 
 7   fact_staff_churn                 2438 non-null   float64 
 8   flag_high_load_lag_1             2438 non-null   int64   
 9   marketing_costs_lag_1            2438 non-null   float64 
 10  fact_couriers_with_shifts_lag_1  2438 non-null   float64 
 11  predicted_staff_value            2438 non-null   int64   
 12  predic

In [162]:
train_final.to_csv("train_final.csv", index=False)

In [163]:
test_final.to_csv("test_final.csv", index=False)